# Homework 3

In [ ]:
from abc import abstractmethod
import dataclasses
import math
import random
import numpy as np

## Extend Path in a Robot Configuration Space


### Utilities

### RRT-Related Utilties
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.


In [ ]:

from typing import Sequence, Tuple, List, Optional, Union

import collections


# Useful geometric data structures and methods.
def euclidean_distance(a, b):
    return math.sqrt((a.x - b.x)**2 + (a.y - b.y)**2)


class Obstacle:

    @abstractmethod
    def collide(self, config):
        ...


@dataclasses.dataclass
class Circle(Obstacle):
    x: float
    y: float
    r: float

    def collide(self, config):
        return euclidean_distance(config, self) < self.r


RobotConfig = collections.namedtuple('RobotConfig', ['x', 'y'])

ExtentType = Tuple[Tuple[float, float], Tuple[float, float]]

@dataclasses.dataclass
class RobotPlanningProblem:
    """A robot planning problem definition contains the configuration space of
    the robot and all obstacles represented as geometries in the configuration
    space.

    In this problem set, we will consider a simple 2D map, where the robot configuration
    space is represented as a tuple (x, y).
    """

    initial: RobotConfig = RobotConfig(-8, -8)
    goal: RobotConfig = RobotConfig(8, 8)
    extent: ExtentType = ((-10, -10), (10, 10))
    obstacles: Sequence[Obstacle] = (Circle(0, 0, 4), Circle(-5, -5, 2),
                                     Circle(3, -4, 3), Circle(-3, 4, 1))

    def sample(self) -> RobotConfig:
        """Generate a random sample in the robot configuration space."""
        return RobotConfig(random.uniform(self.extent[0][0], self.extent[1][0]),
                           random.uniform(self.extent[0][1], self.extent[1][1]))

    def distance(self, a: RobotConfig, b: RobotConfig) -> float:
        """Return the distance between two robot configurations.

        Args:
          a, b: two robot configurations.

        Return:
          distance: the distance between two configurations.
        """
        return math.sqrt((a.x - b.x)**2 + (a.y - b.y)**2)

    def collide(self, x: RobotConfig) -> bool:
        """Test if the input robot configuration collides with some obstacles.

        Args:
          x: the input configuration.

        Returns:
          collision: True if a collision happens between the input configuration and one of the obstacle in the environment.
        """
        return any(obs.collide(x) for obs in self.obstacles)


@dataclasses.dataclass(frozen=False, eq=False)
class RRTNode:
    """Node of a rapidly-exploring random tree.

    Each node is associated with a robot configuration.
    It also keeps track of
      - the parent in the tree.
      - a list of children nodes in the tree.

    If the node itself is the node for an RRT, node.parent will be set to None.
    If the node is a leaf node in an RRT, node.children will be an empty list.
    """

    config: RobotConfig
    children: List['RRTNode'] = dataclasses.field(default_factory=list)
    parent: 'RRTNode' = None

    def add_child(self, other: 'RRTNode') -> 'RRTNode':
        """Register another node as the child of the current node.

        Args:
          other: the other RRTNode.

        Returns:
          self
        """
        self.children.append(other)
        return self

    def attach_to(self, parent: 'RRTNode') -> 'RRTNode':
        """Attach the current node to another node. This function will:

          - set the parent of self to parent.
          - add self to parent.children.

        Args:
          parent: the parent node.

        Returns:
          self
        """
        self.parent = parent
        self.parent.add_child(self)
        return self

    def backtrace(self, config: bool = True) -> List[RobotConfig]:
        """Return the path from root to the current node.

        For example, if the tree is:

            s0 --> s1 -> s2 -> s3 -> s4
            \
              -> s5 -> s6

        Calling s6.backtrace() will return the list: `[s0, s5, s6]`.

        Args:
            config: if True, return a list of robot configurations.
                Otherwise, return the RRTNode list.

        Returns:
            path: the path from root to the current node.
        """
        path = []

        def dfs(x):
            if x.parent is not None:
                dfs(x.parent)
            path.append(x.config if config else x)

        dfs(self)
        return path

    def __repr__(self) -> str:
        return f"RRTNode(config={self.config}, parent={self.parent})"


class RRT:
    """The RRT tree.

    Note that the RRT class does not explicitly manages all nodes in the tree.
    It only keeps track of the root nodes of the tree.
    To get all nodes in an RRT tree, we need to do a tree traversal from the roots.
    We have provided a helper function traverse_rrt_bfs for you to do that.

    There are two useful helper functions in RRT class.
      - extend: which attaches a new node to an existing node in the tree.
      - nearest: which finds the node in the RRT that is most close to the query node.
    """

    def __init__(self, problem: RobotPlanningProblem,
                 roots: Union[RRTNode, Sequence[RRTNode]]):
        """The constructor of RRT takes two parameters:

        Args:
            problem: an instance of the robot planning problem.
                It will define the distance metric in the configuration space.
            roots: a single RRTNode or a list of RRTNode instances as the root(s) of the RRT.
        """
        if isinstance(roots, RRTNode):
            roots = [roots]
        self.roots = list(roots)
        self.problem = problem
        self.size = len(self.roots)

    def extend(self, parent: RRTNode, child_config: RobotConfig) -> RRTNode:
        """Create a new RRTNode from a robot configuration and set its parent.

        Basically, it creates a new RRTNode instance from child_config and attach this
        new node to the `parent` node. We recommend you to use this function
        instead of doing it yourself because it will help you keep track of the size
        of the RRT, which may be useful for your debugging purpose.

        Args:
          parent: the parent node (in the current RRT).
          child_config: the robot configuration of the new node.

        Returns:
          child: the new node.
        """
        child = RRTNode(child_config).attach_to(parent)
        self.size += 1
        return child

    def nearest(self, config: RobotConfig) -> RRTNode:
        """Return the RRTNode in the tree that is closest to the target
        configuration. We strongly suggest you to take a look at the function
        definition to see the example usage of function `traverse_rrt_bfs` and
        `problem.distance`.

        Args:
          config: the query robot config.

        Returns:
          best_node: a node in the RRT that is closest to the query configuration
            based on the distance metric in the configuration space.
        """
        best_node, best_value = None, np.inf
        for node in traverse_rrt_bfs(self.roots):
            distance = self.problem.distance(node.config, config)
            if distance < best_value:
                best_value = distance
                best_node = node

        return best_node


def traverse_rrt_bfs(nodes: Sequence[RRTNode]) -> List[RRTNode]:
    """Run breadth-first search to return all RRTNodes that are descendants of
    the input list of RRTNodes.

    Args:
      nodes: a list of RRTNodes as the seed nodes.

    Returns:
      a list of RRTNodes that are descendants of the input lists (including the input list).
    """
    queue = list(nodes)
    results = []

    while queue:
        x = queue.pop(0)
        results.append(x)
        queue.extend(x.children)

    return results


########## Test-Related Tools for RRT ##########


def check_path_smooth(problem, path, max_diff, eps=1e-5):
    """Check if a path is collision-free and satisfies the max_diff constraint."""
    if path is None or len(path) == 0:
        return False
    for config1, config2 in zip(path[:-1], path[1:]):
        if any(
                abs(c1 - c2) > max_diff + eps
                for c1, c2 in zip(config1, config2)):
            return False
    return all(not problem.collide(config) for config in path)


def check_path_smooth_and_successful(problem: RobotPlanningProblem,
                                     path,
                                     max_diff,
                                     eps=1e-5):
    """Check if a path is collision-free, satisfies the max_diff constraint,
    and connects initial and goal configs."""
    return (path is not None and len(path) > 0 and
            np.allclose(path[0], problem.initial, atol=eps) and
            np.allclose(path[-1], problem.goal, atol=eps) and
            check_path_smooth(problem, path, max_diff, eps=eps))

### RRT Visualization-Related Utilties
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:

import functools


def to_canvas_coord(problem: RobotPlanningProblem, canvas_size: Tuple[int, int],
                    x: float, y: float) -> Tuple[int, int]:
    return (int((x - problem.extent[0][0]) /
                (problem.extent[1][0] - problem.extent[0][0]) * canvas_size[0]),
            canvas_size[1] - int(
                (y - problem.extent[0][1]) /
                (problem.extent[1][1] - problem.extent[0][1]) * canvas_size[1]))


def visualize_map(problem: RobotPlanningProblem,
                  canvas_size: Tuple[int] = (512, 512)):
    """Visualize a 2D environment.

    Args:
      problem: the robot planning problem.
      canvas_size: width and height of the canvas.

    Returns:
      a PIL image of the visualization.
    """
    from PIL import Image, ImageDraw

    image = Image.new('RGB', canvas_size, color='white')
    draw = ImageDraw.Draw(image)

    gen_xy = functools.partial(to_canvas_coord, problem, canvas_size)

    for obs in problem.obstacles:
        if isinstance(obs, Circle):
            AABB = (*gen_xy(obs.x - obs.r, obs.y + obs.r),
                    *gen_xy(obs.x + obs.r, obs.y - obs.r))
            draw.ellipse(AABB, fill='gray', outline='gray')

    return image


def visualize_path(problem: RobotPlanningProblem,
                   path: List[RobotConfig],
                   canvas_size: Tuple[int, int] = (512, 512),
                   show_nodes: bool = True):
    """Visualize a path in a 2D environment. The starting point will be marked
    as blue. The end point will be marked as green.

    Args:
      problem: the robot planning problem.
      path: a list of RobotConfig.
      canvas_size: width and height of the canvas.
      show_nodes: if True, the function will visualize all nodes along the path.

    Returns:
      a PIL image of the visualization.
    """
    from PIL import Image, ImageDraw

    image = visualize_map(problem, canvas_size=canvas_size)
    draw = ImageDraw.Draw(image)

    gen_xy = functools.partial(to_canvas_coord, problem, canvas_size)

    for config1, config2 in zip(path[:-1], path[1:]):
        draw.line(
            (*gen_xy(config1.x, config1.y), *gen_xy(config2.x, config2.y)),
            fill='red')

    if show_nodes:
        for config in path[1:-1]:
            draw.rectangle(
                [
                    *gen_xy(config.x - 0.1, config.y - 0.1),
                    *gen_xy(config.x + 0.1, config.y + 0.1)
                ],
                fill='red',
            )

    draw.rectangle(
        [
            *gen_xy(path[0].x - 0.3, path[0].y - 0.3),
            *gen_xy(path[0].x + 0.3, path[0].y + 0.3)
        ],
        fill='blue',
    )
    draw.rectangle(
        [
            *gen_xy(path[-1].x - 0.3, path[-1].y - 0.3),
            *gen_xy(path[-1].x + 0.3, path[-1].y + 0.3)
        ],
        fill='green',
    )

    return image


def visualize_rrts(problem: RobotPlanningProblem,
                   rrts: List[RRT],
                   canvas_size: Tuple[int, int] = (512, 512),
                   show_nodes: bool = True):
    """Visualize RRTs in a 2D environment. The initial config will be marked as
    blue. The goal config will be marked as green.

    Args:
      problem: the robot planning problem.
      rrts: a list of RRTs. Currently the function supports up to two rrts. The first one will show in red,
        while the second one wil show in blue.
      canvas_size: width and height of the canvas.
      show_nodes: if True, the function will visualize all nodes in the tree.

    Returns:
      a PIL image of the visualization.
    """
    from PIL import ImageDraw

    image = visualize_map(problem, canvas_size=canvas_size)
    draw = ImageDraw.Draw(image)

    gen_xy = functools.partial(to_canvas_coord, problem, canvas_size)

    for rrt, color in zip(rrts, ['red', 'blue']):
        for node in traverse_rrt_bfs(rrt.roots):
            if node.parent is not None:
                draw.line((*gen_xy(node.parent.config.x, node.parent.config.y),
                           *gen_xy(node.config.x, node.config.y)),
                          fill=color)

            if show_nodes:
                draw.rectangle(
                    [
                        *gen_xy(node.config.x - 0.1, node.config.y - 0.1),
                        *gen_xy(node.config.x + 0.1, node.config.y + 0.1)
                    ],
                    fill='red',
                )

    draw.rectangle(
        [
            *gen_xy(problem.initial.x - 0.3, problem.initial.y - 0.3),
            *gen_xy(problem.initial.x + 0.3, problem.initial.y + 0.3)
        ],
        fill='blue',
    )
    draw.rectangle(
        [
            *gen_xy(problem.goal.x - 0.3, problem.goal.y - 0.3),
            *gen_xy(problem.goal.x + 0.3, problem.goal.y + 0.3)
        ],
        fill='green',
    )

    return image

### Question

In this section, you will implement an important utility for doing motion planning in a continuous space: `extend_path`.
The function takes in a RobotPlanningProblem. It makes a linear interpolation between the start and end
configurations in the robot's configuration space.

Its output is a path, that is, a list of robot configurations along the line from `start` to `end`.
When the `start` and `end` cannot be directly connected with a line because of collision,
your function should return a "prefix" of the path.
In other words, you should try to move towards `end` until you reach some obstacles in the environment.

Take a close look at the docstring of the `RobotPlanningProblem` class before you implement this function.

Note that:
- Although our test cases are for a two-dimensional configuration $[x, y]$.  
    The code for `extend_path` is written so that it can work for any number of dimensions.
    Furthermore, the implementation of RRT should also work for any dimension without change.
- Our solution does not use `numpy`. However, if you are comfortable with `numpy`,
    feel free to use it to shorten your implementation.


For reference, our solution is **18** line(s) of code.

In [ ]:
def extend_path(problem: RobotPlanningProblem, start: RobotConfig,
                end: RobotConfig,
                max_diff: float) -> Tuple[bool, List[RobotConfig]]:
    """Construct a linear path in the configuration space from the initial
    configuration to the goal configuration.

    Your implementation should have two steps.
    Step 1, construct a path that is a linear interpolation between `start` and `end`.
    At each step, the movement along x and along y should be both smaller than or equal to `max_diff`.

    Step 2: check for collisions.
    You should iterate over the configurations in the path generated in step 1,
    if a configuration triggers a collision with an obstacle,
    your algorithm should return the longest prefix path that does not have any collision.
    We have implemented this test for you. But make sure you read the code and understand how it works.

    Note that, even though we have separated linear interpolation and collision checking as two
    separate steps, you can also choose to do them simultaneously in a single for loop.

    Args:
      problem: a RobotPlanningProblem instance, used for collision checking.
      start: a starting RobotConfig. It should be the first element of your return list.
      end: an ending RobotConfig. It should be the last element of your return list (if succeed).
      max_diff: the maximum movement in x or y.

    Returns:
      successful: a bool value indicating whether we have found a path from the initial configuration to the goal configuration.
        If a collision happens, you should return False. See the starter code.
      path: a list of RobotConfig: extending from the initial to the goal configuration.
    """
    path = [start]
    # Step 1: Generate a list of robot configurations that connects `start` and `goal`.
    # Your code only needs to take care of max_diff. You can ignore collision checking for now.
    # Your code here...
    assert path[-1] == end

    # Step 2: Collision checking.
    for i, config in enumerate(path):
        if problem.collide(config):
            return False, path[:i]
    return True, path

### Tests

In [ ]:
def extend_path_test1():
    problem = RobotPlanningProblem()
    success, path = extend_path(problem, problem.initial, problem.goal, 0.1)
    assert not success  # you can't reach the goal with a single linear motion.
    assert len(path) > 10  # you should make some progress.
    assert check_path_smooth(problem, path, 0.1)

extend_path_test1()


def extend_path_test2():
    problem = RobotPlanningProblem(RobotConfig(-8, -8), RobotConfig(-7, 6))
    success, path = extend_path(problem, problem.initial, problem.goal, 0.1)
    assert success
    assert check_path_smooth_and_successful(problem, path, 0.1)

extend_path_test2()

print('Tests passed.')

## Extend Path Visualization


### Question

We have created a visualization tool for you in colab to visualize the path.
You can run the code blocks and they will generate the visualized map and the path.


### Tests

In [ ]:
def extend_path_visualization():
    from IPython.display import display
    problem = RobotPlanningProblem(RobotConfig(-8, -8), RobotConfig(-7, 6))
    # we have set max_diff to 0.5 for better visualization.
    success, path = extend_path(problem, problem.initial, problem.goal, 0.5)
    display(visualize_path(problem, path))

extend_path_visualization()

print('Tests passed.')

## RRT


### Question

In this section, you will implement the RRT algorithm we talked about in
class to find a path from one configuration to the goal configuration
in a 2D environment.

We have implemented some utility classes (RRTNode, RRT) for you,
as well as their corresponding visualization tools.
You should take a close look at their docstrings before you implement
this function. In particular, RRT.\_\_init\_\_, RobotPlanningProblem.sample, RRT.extend, RRT.nearest, RRTNode.backtrace should be helpful. **To run in Colab, you will need an implementation of `extend_path`,
if you did not write your own, use the solution in Catsoop.**

**Hint**: Recall that there is a hyperparameter in RRT, which balances
how frequently we want to test if the goal configuration can be reached
from a node in the current RRT tree. Our suggested value for this
hyperparameter is $0.1$.


For reference, our solution is **19** line(s) of code.

In addition to all the utilities defined at the top of the Colab notebook, the following functions are available in this question environment: `extend_path`. You may not need to use all of them.

In [ ]:
def rrt(problem: RobotPlanningProblem,
        max_diff: float,
        nr_iterations: int = 100,
        goal_sample_prob: float = 0.1) -> Tuple[List[RobotConfig], RRT]:
    """Implement RRT algorithm to compute a path from the initial configuration
    to the target configuration. Your implementation should returns a list of
    robot configurations, starting from `problem.initial` and terminating at `proble.goal`.

    Similar to the definition in extend_path, at each step, the movement along x and along y should be both smaller than or equal to max_diff.
    You should use the extend_path utility that you just implemented to check whether there is a path from one node to another.

    Args:
      problem: an RobotPlanningProblem instance, used for configuration sampling and collision checking.
        `problem.initial` should be the first element of your return list.
        `problem.goal` should be the last element of your return list.
      max_diff: the maximum movement in x or y.
      nr_iterations: the number of iterations in RRT.
      goal_sample_prob: at each iteration, with prob p, you should try to construct a path from a node to the goal configuration.
        Otherwise, you should sample a new configuration and try to constrcut a path from a node to it.

    Returns:
      path: a list of RobotConfig as the trajectory. Return None if we can't find such a path after the specified number of iterations.
      rrt: an RRT object. The root should be the intial_config. There must also exist another node for goal_config.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def rrt_test():
    problem = RobotPlanningProblem(RobotConfig(-8, -8), RobotConfig(8, 8))
    path, tree = rrt(problem, 0.5, nr_iterations=200)
    assert check_path_smooth_and_successful(problem, path, 0.5)

rrt_test()

print('Tests passed.')

## RRT Visualization


### Question

We have created a visualization tool for you in colab to visualize the RRT tree during your search. You can run the following code blocks
and they will generate the visualized the map as well as some lines showing the edges in the RRT.

If you run this cell multiple times, you will see different RRT trees due to different random seeds.
You will find that sometimes the tree has very few nodes -- indicating that your RRT algorithm finds a solution very quickly.
Sometimes it takes many iterations to find the solution. This suggests that the algorithm is very sensitive to random seeds,
which is bad for practical usage. Thus, in practice, we shuold always use improved RRT algorithm variants such as BiRRT (bidirectional RRT).

We recommend you try in your colab notebook to see how the hyperparameter `goal_sample_probability` will affect the search process.
Specifically, try to set that probability to $0.05$ or $0.2$ and visualize your result.


### Tests

In [ ]:
def rrt_visualization():
    from IPython.display import display
    problem = RobotPlanningProblem()
    # we have set max_diff to 0.5 for better visualization.
    path, tree = rrt(problem, 0.5)
    display(visualize_rrts(problem, [tree]))

rrt_visualization()

print('Tests passed.')

## Numerical Integrator Warmup


### Utilities

Trajectory Optimization Problem Abstractions
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:

from typing import Dict, Tuple, Sequence, Union, Optional

Point2D = np.ndarray  # shape == (2, )


@dataclasses.dataclass
class TrajOpt2DProblem:
    """Base class for a trajectory optimization problem in R^2. """

    initial: Point2D = (-8, -8)
    goal: Point2D = (8, 8)

    def __post_init__(self):
        self.initial = np.array(self.initial)
        self.goal = np.array(self.goal)

    def cost(self, coord: Tuple[float, float]) -> float:
        """Returns the cost at single coord."""
        assert len(coord) == 2
        return self.cost_field(np.array(coord)).item()

    @abstractmethod
    def cost_field(self, coords: np.ndarray) -> np.ndarray:
        """The costs at `coords`. Handles a batch of coordinates for efficiency.

        Subclass must override to provide a meaningful cost field.

        Args:
            coords: an array of shape (2,) or (...batch, 2) denoting the coordinate(s)
                to compute the cost field.

        Returns:
            A scalar or an array of shape (...batch,)
        """
        ...

    def visualize_cost_field(self,
                             fov=((-10, -10), (10, 10)),
                             resolution=(512, 512),
                             show=False):
        """Visualize a cost field.

        Args:
            fov: a field of view for the visualization.
            resolution: number of pixels (w, h) to discretize the field of view.
            show: whether to display the plot immediately.
        """
        import matplotlib.pyplot as plt
        plt.xlim(fov[0][0], fov[1][0])
        plt.ylim(fov[0][1], fov[1][1])
        coords = np.dstack(
            np.meshgrid(
                np.linspace(fov[0][0], fov[1][0], resolution[0]),
                np.linspace(fov[0][1], fov[1][1], resolution[1]),
            )).reshape(-1, 2)
        vals = self.cost_field(coords).reshape(resolution)
        plt.imshow(vals,
                   origin="lower",
                   extent=(fov[0][0], fov[1][0], fov[0][1], fov[1][1]))
        plt.plot(*self.initial, 'o')
        plt.plot(*self.goal, 'o')
        plt.colorbar()
        if show:
            plt.show()
        return self

    def visualize_trajectory(self, trajectory, show=False, **arrow_kwargs):
        """Visualize a piecewise linear trajectory on the cost field.

        Visualizes a plan on top of the cost field.
        Usually, one calls `visualize_cost_field` first to draw the cost
        field in the back.
        One then chain multiple `visualize_trajectory` calls, and pass `show=True` to the
        last call to visualize multiple trajectories on the cost field.

        Args:
        trajectory: a piecewise linear trajectory.
        arrow_kwargs: passed to `plt.arrow`.
        show: whether to display the plot immediately.
        """
        import matplotlib.pyplot as plt
        for s1, s2 in zip(trajectory[:-1], trajectory[1:]):
            x1, y1 = s1
            x2, y2 = s2
            dx, dy = x2 - x1, y2 - y1
            plt.arrow(x1,
                      y1,
                      dx,
                      dy,
                      length_includes_head=True,
                      width=0.01,
                      fill=True,
                      **arrow_kwargs)
        if show:
            plt.show()
        return self


import scipy.stats


@dataclasses.dataclass
class GMTrajOpt2DProblem(TrajOpt2DProblem):
    """A cost field with a mixture of guassian gradients.

    Args:
        locs: the centers of the gaussians
        covs: a scalar, a sequence of scalars, or a sequence of matrices
            for the covariances of the gaussians.
        strengths: a scalar or a sequence of scalars for the scalaring factors
            for each guassian.
    """
    locs: Sequence[Tuple[int, int]] = ((0, 0),)
    covs: Union[float, Sequence[Union[float, np.ndarray]]] = 0.1
    strengths: Union[float, Sequence[float]] = 1.

    def __post_init__(self):
        self.locs = np.array(self.locs)
        if np.isscalar(self.covs):
            self.covs = [self.covs] * len(self.locs)
        self.covs = [
            np.eye(2) * cov if np.isscalar(cov) else cov for cov in self.covs
        ]
        self.covs = np.array(self.covs)
        if np.isscalar(self.strengths):
            self.strengths = [self.strengths] * len(self.locs)
        self.strengths = np.array(self.strengths)
        # Sanity checks
        k = self.locs.shape[0]
        assert (self.locs.shape == (k, 2) and self.covs.shape == (k, 2, 2) and
                self.strengths.shape == (k,))

    def cost_field(self, coords):
        coords = np.atleast_2d(coords)
        return sum(
            scipy.stats.multivariate_normal.pdf(coords, mu, sigma) * strength
            for mu, sigma, strength in zip(self.locs, self.covs,
                                           self.strengths))


@dataclasses.dataclass
class Circle:
    x: float
    y: float
    r: float


@dataclasses.dataclass
class RobotPlanningTrajOpt2DProblem(TrajOpt2DProblem):
    """A trajectory optimization problem similar to `RobotPlanningProblem`.

    This problem only deals with round obstacles (i.e., `Circle`s).

    For any point $p$ and obstacle $o$,
    the signed distance field at point $p$ has magnitude of the shortest distance
    from $p$ to $o$'s boundary. The signed distance field is positive inside the
    obstacle and negative outside (therefore "signed").

    Then, for any point $p$, the cost field of this problem is the maximum value
    of the signed distance field at point $p$ induced by all obstacles.

    Note that, for added safety (to avoid collisions), for each obstacle, we
    "enlarge" its radius by `margin` amount when computing the cost field.
    """
    obstacles: Sequence[Circle] = (
        Circle(0, 0, 4),
        Circle(-5, -5, 2),
        Circle(3, -4, 3),
        Circle(-3, 4, 1),
    )
    margin: float = 0.1

    def __post_init__(self):
        self.obstacles = list(self.obstacles)
        self._obstacle_locs = np.array([[obs.x, obs.y] for obs in self.obstacles
                                       ])
        self._obstacle_radius = np.array([obs.r for obs in self.obstacles])

    def cost_field(self, coords):
        """The cost at $x$ is given by:
        $$ \max_{obs \in obstacles} \max(0, margin + r - |(obs.x, obs.y) - x|)$$
        """
        coords = np.atleast_2d(coords)
        dists = np.linalg.norm(coords[..., np.newaxis, :] - self._obstacle_locs,
                               axis=-1)
        return np.max(
            np.maximum(self.margin + self._obstacle_radius - dists, 0),
            axis=-1,
        )

    def visualize_cost_field(self, *args, **kwargs):
        import matplotlib.pyplot as plt
        ax = plt.gca()
        for obs in self.obstacles:
            circ = plt.Circle((obs.x, obs.y), obs.r, color='gray', fill=False)
            ax.add_patch(circ)
        return super().visualize_cost_field(*args, **kwargs)


def get_traj_opt_problems(
    *name: Optional[Union[str, Sequence[str]]]
) -> Union[Dict[str, TrajOpt2DProblem], TrajOpt2DProblem]:
    """A few predefined trajectory optimiztation problems.

    DO NOT CHANGE THIS FUNCTION --- it exists to protect you from accidentally
    changing the problems.
    """
    problems = {
        # A simple skewed 2D Gaussian cost field
        "gm-1":
            lambda: GMTrajOpt2DProblem(
                locs=[[0, 1]],
                covs=[[[12, -12], [-12, 24.]]],
                strengths=[500.],
            ),
        # A problem with a _negated_ cost field, which is equivalent to
        # maximizing over a reward field.
        "gm-2":
            lambda: GMTrajOpt2DProblem(
                locs=[(-6, 6), (6, -6)],
                covs=[10, 30],
                strengths=[-200, -200.],
            ),
        # A simple obstacle avoidance problem
        "robot-planning-1":
            lambda: RobotPlanningTrajOpt2DProblem(obstacles=[
                Circle(-2, -2, 5),
            ]),
        # A simple obstacle avoidance problem where the optimizer can be
        # tricked into a local minima
        "robot-planning-2":
            lambda: RobotPlanningTrajOpt2DProblem(obstacles=[
                Circle(-2.01, -2, 5.2),
                Circle(7, 2.1, 2.2),
            ]),
        # An obstacle avoidance problem similar to the example for RRT.
        "robot-planning-3":
            lambda: RobotPlanningTrajOpt2DProblem(),
    }
    if len(name) == 1:
        return problems[name[0]]()
    if len(name) == 0:
        name = problems.keys()
    return {n: problems[n]() for n in name}

The integrator interface and some closed-form integrators
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:

import functools

LineSeg = np.ndarray  # shape == (2, 2)
PiecewiseLinearTraj = np.ndarray  # shape == (k, 2), where k is the number of waypoints


class LineSegCostIntegrator:
    """A callable inteface to compute the path-integral of a cost field over
    a line segment.

    The input line segment is represented by `LineSeg`, which is a shape (2, 2)
    array representing two points.
    In particular, if lineseg is of type `LineSeg`, then `lineseg[0]` is the
    first point and `lineseg[1]` is the second point.

    Note that we use Python's `__call__` feature to turn instances of
    (subclass of) this class into callables.
    See https://www.geeksforgeeks.org/__call__-in-python/ for an explanation of
    this feature.
    """

    @abstractmethod
    def __call__(self, lineseg: LineSeg) -> float:
        ...


def sqrtm2(mat: np.ndarray) -> np.ndarray:
    """Computes the square root of a 2 by 2 symmetric positive definite matrix.
    Formula from: https://en.wikipedia.org/wiki/Square_root_of_a_2_by_2_matrix.

    Args:
        mat: array of shape [...batch, 2, 2]; a SPD matrix or matrices.

    Returns:
        an array of shape [...batch, 2, 2]; the square root(s) of the input(s).
    """
    s = np.sqrt(np.linalg.det(mat))
    t = np.sqrt(2 * s + np.trace(mat, axis1=-2, axis2=-1))
    return (mat + s[:, np.newaxis, np.newaxis] *
            np.eye(2, dtype=mat.dtype)) / t[:, np.newaxis, np.newaxis]


@dataclasses.dataclass
class GMTrajOpt2DIntegrator:
    """A closed form integrator for mixture of Gaussians cost field."""

    problem: GMTrajOpt2DProblem

    def __post_init__(self):
        """Precompute some quantities related to sigma."""
        # For below, k == len(problem.locs)
        self.sigma_inv_2s = np.linalg.inv(sqrtm2(
            self.problem.covs))  # shape (k, 2, 2)
        self.sigma_inv_2_dets = np.linalg.det(self.sigma_inv_2s)  # shape (k,)

    def __call__(self, lineseg: LineSeg) -> float:
        orig_seglen = np.linalg.norm(lineseg[0] - lineseg[1])
        if orig_seglen == 0:
            return 0
        # Transform input lineseg into spaces where the Gaussians become N((0,0), I)
        transformed_lineseg = np.matmul(
            lineseg - self.problem.locs[:, np.newaxis, :], self.sigma_inv_2s)
        # Extract the start and end point of the transformed line segments
        a, b = transformed_lineseg[:, 0], transformed_lineseg[:, 1]
        # Compute lengths of the transformed segments
        seglen = np.linalg.norm(a - b, axis=-1)  # shape (k, )
        # Helper for batch vector-vector dot product
        bvdot = functools.partial(np.einsum, "bi,bi->b")
        return np.sum(
            scipy.stats.norm.pdf(
                a[:, 1] * b[:, 0] - a[:, 0] * b[:, 1], loc=0, scale=seglen) *
            np.abs(
                scipy.stats.norm.cdf(bvdot(b, b - a), loc=0, scale=seglen) -
                scipy.stats.norm.cdf(bvdot(a, b - a), loc=0, scale=seglen)) *
            self.sigma_inv_2_dets * self.problem.strengths,
            axis=0,
        ) * orig_seglen


def integral_point2line_distance(a: np.ndarray, b: np.ndarray,
                                 x: float) -> np.ndarray:
    """Computes the integral of Euclidean distance from a point to the x-axis.

    Given a point (a, b) in the first quadrant, compute the (indefinite) integral
    of the Euclidean distance to the x-axis:
                ⌠    ________________
                ⎮   ╱  2          2
                ⎮ ╲╱  b  + (x - a)     dx
                ⌡

    The function levarages numpy broadcasting to allow taking in `a` and
    `b` as batches of points.

    Args:
        a: array of shape (...batch,)
        b: array of shape (...batch,)
        x: float for where to evaluate the indefinite integral

    Returns:
        the evaluation of the above indefinite integral(s) at `x`.
    """
    l = np.sqrt(b**2 + (a - x)**2)
    return ((x - a) * l - b**2 * np.log(np.maximum(a + l - x, 1e-12))) / 2


@dataclasses.dataclass
class RobotPlanningTrajOpt2DIntegrator:
    """A closed-form integrator for the RobotPlanningTrajOpt2DProblem."""

    problem: RobotPlanningTrajOpt2DProblem

    def __post_init__(self):
        self.obs_locs = np.array(
            [[obs.x, obs.y] for obs in self.problem.obstacles])
        self.obs_r = np.array(
            [obs.r + self.problem.margin for obs in self.problem.obstacles])

    def __call__(self, lineseg: LineSeg) -> float:
        a, b = lineseg
        # Move coordinate system so that the origin is at `a`
        u = b - a
        obs_locs = self.obs_locs - a
        u_norm = np.linalg.norm(u)
        if u_norm == 0:
            return 0
        # Rotate u to align with the unit vector along x-axis: [1, 0]
        ux, uy = u
        rot_mat = np.array([[ux, uy], [-uy, ux]]) / u_norm
        # We now have: u_rot == np.array([0, np.linalg.norm(u)])
        obs_locs_rot = np.einsum("ij,bj->bi", rot_mat, obs_locs)
        orx, ory = obs_locs_rot[:, 0], obs_locs_rot[:, 1]
        # Flip the coordinate so that the obstacle lies in the first quadrant
        ory = np.abs(ory)
        orx = np.where(orx < 0, u_norm - orx, orx)
        # Compute endpoints of effective line segment overlapping with the obstacle
        with np.errstate(invalid='ignore'):
            intercept1 = orx - np.sqrt(self.obs_r**2 - ory**2)
            intercept2 = orx + np.sqrt(self.obs_r**2 - ory**2)
        p0 = np.maximum(intercept1, 0)
        p1 = np.minimum(intercept2, u_norm)
        has_obs_intercetion = np.logical_and(ory < self.obs_r, p0 < p1)
        # Compute the integral
        return np.max(
            np.where(
                has_obs_intercetion,
                integral_point2line_distance(orx, ory, p0) -
                integral_point2line_distance(orx, ory, p1) + self.obs_r *
                (p1 - p0),
                0,
            ))


def get_integrator(problem: TrajOpt2DProblem) -> LineSegCostIntegrator:
    """Helper to returns a suitable integrator for the problem.

    If `problem` is recognizable, return an efficient closed-form
    integrator. Otherwise, default to a numerical integrator.
    """
    if isinstance(problem, GMTrajOpt2DProblem):
        return GMTrajOpt2DIntegrator(problem)
    if isinstance(problem, RobotPlanningTrajOpt2DProblem):
        return RobotPlanningTrajOpt2DIntegrator(problem)
    # You must implement LineSegCostNumericalIntegrator for this fallback to work
    return LineSegCostNumericalIntegrator(problem)

### Question

In this problem, you will implement an integrator for a general `TrajOpt2DProblem`
using numerical integration.

Given a `TrajOpt2DProblem` instance, we will use
[numerical integration](https://en.wikipedia.org/wiki/Numerical_integration)
implemented in [scipy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.quad.html)
to approximately compute the path integral over the cost field of the problem.
The path integral follows the formula as in AIMA (Section 26.5, P949).
Note that since our robot is a point, the inner integration over all
points of the robot's body is unnecessary. So our formula will be a simpler
version of the one in AIMA.

Given the two endpoints $a$ and $b$ of a line segment, we parameterize the
line segment as $\tau(t) = (b - a) \cdot t + a$ for $t \in [0, 1]$.
The path-integral of cost $c$ over the line segment is then:
    \begin{align}
    \int_{0}^{1} c(\tau(t)) \|\tau'(t)\| dt
    &= \int_{0}^{1} c\left(\left(b - a\right) \cdot t + a\right) \cdot \left\|b - a\right\| \cdot dt  \\
    &= \|b - a\| \int_{0}^{1} c\left(\left(b - a\right)\cdot t + a\right) dt
    \end{align}

In words, the path integral of the line segment is the segment's length times
the integral of the value of cost field over the line segment.

For this problem, we have implemented most of the code for you.
_You should only need to fill in the `integrand` function in the template._


For reference, our solution is **10** line(s) of code.

In [ ]:
@dataclasses.dataclass
class LineSegCostNumericalIntegrator(LineSegCostIntegrator):
    """A generic line segment cost function for any `TrajOpt2DProblem`.
    """
    problem: TrajOpt2DProblem

    def __call__(self, lineseg: LineSeg) -> float:
        a, b = lineseg

        def integrand(t):
            """Evaluate `self.problem`'s cost field at point tau(t) = (b - a) * t + a."""
            # YOUR CODE HERE #
            return 0

        seglen = np.linalg.norm(a - b)
        return scipy.integrate.quad(integrand, 0, 1)[0] * seglen

### Tests

In [ ]:
def numerical_integration_test1(LineSegCostNumericalIntegrator, lineseg,
                                traj_opt_prob_name):
    lineseg = np.array(lineseg)
    traj_opt_problem = get_traj_opt_problems(traj_opt_prob_name)
    numerical_integrator = LineSegCostNumericalIntegrator(traj_opt_problem)
    closedform_integrator = get_integrator(traj_opt_problem)

    ci_result = closedform_integrator(lineseg)
    ni_result = numerical_integrator(lineseg)
    assert np.isclose(
        ci_result, ni_result
    ), f"{ni_result} does not match with {ci_result} for integrator: {closedform_integrator} input: {lineseg}"

numerical_integration_test1(LineSegCostNumericalIntegrator, ((0, 0), (1, 1)), "gm-1")

numerical_integration_test1(LineSegCostNumericalIntegrator, ((0, 0), (1, 1)), "gm-2")

numerical_integration_test1(LineSegCostNumericalIntegrator, ((0, 0), (1, 1)), "robot-planning-1")

numerical_integration_test1(LineSegCostNumericalIntegrator, ((0, 0), (-1, -1)), "gm-1")

numerical_integration_test1(LineSegCostNumericalIntegrator, ((0, 0), (-1, -1)), "gm-2")

numerical_integration_test1(LineSegCostNumericalIntegrator, ((0, 0), (-1, -1)), "robot-planning-1")

numerical_integration_test1(LineSegCostNumericalIntegrator, ((0, 0), (0, 0)), "gm-1")

numerical_integration_test1(LineSegCostNumericalIntegrator, ((0, 0), (0, 0)), "gm-2")

numerical_integration_test1(LineSegCostNumericalIntegrator, ((0, 0), (0, 0)), "robot-planning-1")

numerical_integration_test1(LineSegCostNumericalIntegrator, ((0, 0), (1, 0)), "gm-1")

numerical_integration_test1(LineSegCostNumericalIntegrator, ((0, 0), (1, 0)), "gm-2")

numerical_integration_test1(LineSegCostNumericalIntegrator, ((0, 0), (1, 0)), "robot-planning-1")

numerical_integration_test1(LineSegCostNumericalIntegrator, ((1, 0), (2, 0)), "gm-1")

numerical_integration_test1(LineSegCostNumericalIntegrator, ((1, 0), (2, 0)), "gm-2")

numerical_integration_test1(LineSegCostNumericalIntegrator, ((1, 0), (2, 0)), "robot-planning-1")

numerical_integration_test1(LineSegCostNumericalIntegrator, ((1.4142135623730951, 1.7320508075688772), (2.6457513110645907, 2.0)), "gm-1")

numerical_integration_test1(LineSegCostNumericalIntegrator, ((1.4142135623730951, 1.7320508075688772), (2.6457513110645907, 2.0)), "gm-2")

numerical_integration_test1(LineSegCostNumericalIntegrator, ((1.4142135623730951, 1.7320508075688772), (2.6457513110645907, 2.0)), "robot-planning-1")
print('Tests passed.')

## Trajectory Optimization


### Question

Implement trajectory optimization `traj_opt`.

The `traj_opt` function finds a piecewise linear trajectory by performing
optimization.
It minimizes the sum of two terms $J_\textit{cost\_field}$ and $J_\textit{eff}$
and trades off between the two using $\lambda_\textit{eff}$:
    $$J = J_\textit{cost\_field} + \lambda_\textit{eff} \cdot J_\textit{eff}$$

For our setup:

- $J_\textit{eff} = \sum_{p \in \text{segments} } \| p \| $ is the total length of the trajectory.
- $J_\textit{cost\_field} = \sum_{p \in \text{segments}} \texttt{lineseg\_costf}(p) $ is the sum of the costs returned by
    `lineseg_costf`, for all line segments in the piecewise linear trajectory.

We will use the [BFGS optimizer](https://en.wikipedia.org/wiki/Broyden–Fletcher–Goldfarb–Shanno_algorithm)
implemented in `scipy` to carry out the minimization.

Here's a rough overview of the algorithm:
1. Create an initial path with `num_waypoints` number of intermediate points by
    linearly interpolating between the initial configuration and the goal configuration,
    This step is similar to `extend_path`.
1. "Slice" the initial path to contain only the intermediate way points --- these are
    the parameters of the optimization problem.
1. Add a small random noise perturbation to the initial parameters. (What potential problem are we trying to solve by doing this?)
1. Build the cost functions $J_\textit{cost\_field}$, $J_\textit{eff}$ and $J$.
1. Minimize $J$ using `scipy.optimize.minimize`.
1. Recover the complete trajectory from `scipy`'s optimization result.

_Hint: the following `numpy` functions might be quite handy (though not strictly necessary):_
[np.linspace](https://numpy.org/doc/stable/reference/generated/numpy.linspace.html),
[np.random.normal](https://numpy.org/doc/stable/reference/random/generated/numpy.random.normal.html),
[np.stack](https://numpy.org/doc/stable/reference/generated/numpy.stack.html),
[np.vstack](https://numpy.org/doc/stable/reference/generated/numpy.vstack.html),
[np.reshape](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html),
[np.flatten](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.flatten.html).
In general, leveraging `numpy` could make your life easier! If you get stuck or unsure how to use these functions,
please feel free to reach out and we will be happy to help!

**Note** Sometimes `scipy` might return a result with a `success=False` flag.
For our example problems, one common reason of this `success=False` flag is the
loss of accuracy during the optimization.
Usually, this is fine and `scipy` will still return a fairly good solution.
However, if you do run into wierd results and get stuck, please reach out!


For reference, our solution is **36** line(s) of code.

In addition to all the utilities defined at the top of the Colab notebook, the following functions are available in this question environment: `LineSegCostNumericalIntegrator`. You may not need to use all of them.

In [ ]:
def traj_opt(
    initial: Point2D,
    goal: Point2D,
    lineseg_costf: LineSegCostIntegrator,
    num_waypoints: int = 5,
    lambda_eff: float = 10.,
    init_perturb: float = 1e-2
) -> Tuple[PiecewiseLinearTraj, float, float, bool]:
    """Performs trajectory optimization with a given cost function.

    This function finds a piecewise linear trajectory by performing optimization.
    It minimizes the sum of a J_cost_field and J_eff (and trades off between
    the two):
        J = J_cost_field + lambda_eff * J_eff

    For our setup:
        - `J_eff` is the total lengths of all pieces of the trajectory.
        - `J_cost_field` is the sum of the costs returned by `lineseg_costf`,
            for all line segments in the piecewise linear trajectory.

    We will use the BFGS optimizer implemented in scipy to carry out the
    minimization.

    Args:
        initial: the initial point of the trajectory.
        goal: the goal point of the trajectory.
        lineseg_costf: a cost function for each line segment in a piecewise
            linear trajectory.
        num_waypoints: number of intermediate points for the piecewise linear
            trajectory (excluding `initial` and `goal`). Must be >= 0.
        lambda_eff: a weighing factor for J_eff, as in AIMA.
        init_perturb: this function adds a normally distributed random noise to
            the initial guess. The noise has zero mean. This parameter controls
            the variance.

    Returns:
        - the optimal piecewise linear trajectory.
        - the value of J_cost_field of the final trajectory.
        - the value of J_eff of the final trajectory.
        - success: whether the optimization converged. See
            https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.OptimizeResult.html
            on how to access the optimization result.
    """
    # Interpolate from initial to goal
    init_traj = ...  # YOUR CODE HERE
    # Remove initial and goal and flatten the trajectory to shape (num_waypoints*2, )
    init_traj_params = ...  # YOUR CODE HERE
    # Add random noise perturbation
    init_traj_params += np.random.normal(loc=0,
                                         scale=init_perturb,
                                         size=init_traj_params.shape)

    def to_complete_trajectory(traj_params):
        """Given a set of trajectory parameters of shape (num_waypoints * 2,),
        return the complete trajectory of shape (num_waypoints+2, 2) that
        includes the initial and goal."""
        return np.vstack([initial, traj_params.reshape(num_waypoints, 2), goal])

    def J_cost_field(traj: np.ndarray) -> float:
        """Use `lineseg_costf` to compute the total cost of the trajectory."""
        # YOUR CODE HERE #

    def J_eff(traj: np.ndarray) -> float:
        """Compute the total length of the trajectory."""
        # YOUR CODE HERE #

    def J(traj_params: np.ndarray) -> float:
        """Compute the trajectory cost as: J = J_cost_field + lambda_eff * J_eff"""
        # YOUR CODE HERE #

    # Call the optimizer: we have filled in the arguments for you
    result = scipy.optimize.minimize(
        J,
        init_traj_params,
        method="BFGS",
        options=dict(
            gtol=1e-4,
            maxiter=int(1e6),
        ),
    )
    # The parameters found by running the optimizer
    found_traj_params = result.x

    # Convert parameters back into a complete trajectory
    traj = ...  # YOUR CODE HERE #

    return traj, J_cost_field(traj), J_eff(traj), result.success

### Tests

In [ ]:
def traj_opt_test(traj_opt, prob_name, num_waypoints, lambda_eff, results):
    """The tests here are not thorough.

    Instead, we will ask you to run the optimizations on some problems and
    test your understanding of trajectory optimization.

    To make sure you have the correct implementation, run `traj_opt` on more of
    our predefined problems in `get_traj_opt_problems` to check if it matches
    your expectation. As always, reach out if you get stuck!
    """
    traj_opt_prob = get_traj_opt_problems(prob_name)
    integrator = get_integrator(traj_opt_prob)

    traj, J_cost, J_eff, success = traj_opt(traj_opt_prob.initial,
                                            traj_opt_prob.goal,
                                            integrator,
                                            num_waypoints=num_waypoints,
                                            init_perturb=1e-3,
                                            lambda_eff=lambda_eff)
    assert traj.shape == (num_waypoints + 2, 2)
    assert np.allclose(traj[0], traj_opt_prob.initial)
    assert np.allclose(traj[-1], traj_opt_prob.goal)

    r_J_cost, r_J_eff, r_success = results
    assert success == r_success
    assert np.allclose(J_cost, r_J_cost, atol=0.5)
    assert np.allclose(J_eff, r_J_eff, atol=0.5)

traj_opt_test(traj_opt, "gm-1", 6, 1.0, (4.594336331662315, 33.25032307982191, True))

traj_opt_test(traj_opt, "robot-planning-1", 6, 0.5, (0.00011048369573735117, 25.16947065779389, True))
print('Tests passed.')

## Trajectory Optimization Visualization


### Question

We have created a visualization tool for you in colab to visualize the result of your `traj_opt`.
You can run the code blocks and they will generate the visualized cost field and the found trajectory.

As you do the visualization, try to:
- Visualize the different problems in `get_traj_opt_problems`: `gm-1`,`gm-2`,`robot-planning-1`,`robot-planning-2`,`robot-planning-3`
- Vary the parameter $\lambda_{\textit{eff}}$ --- are larger or smaller values more likely to produce an obstacle-avoiding path in `robot-planning-*` problems?


### Tests

In [ ]:
def visualize_traj_opt():
    # Change this line to visualizer other problems
    problem_name = "gm-1"  #@param {type:"string"}
    problem = get_traj_opt_problems(problem_name)
    integrator = get_integrator(problem)

    # Uncomment line below if you want to run with your own numerical integrator
    # It should produce the same result (modulo numerical accuracy differences)
    # as using our integrator, but likely runs slower.
    # integrator = LineSegCostNumericalIntegrator(problem)

    # Run the optimization
    num_waypoints = 6  #@param {type:"integer"}
    lambda_eff = 2.  #@param {type:"number"}
    traj, *_ = traj_opt(problem.initial,
                        problem.goal,
                        integrator,
                        num_waypoints=num_waypoints,
                        lambda_eff=lambda_eff)
    view_size = 20  # {type:"slider", min:10, max:100, step:1}
    view_size2 = view_size // 2
    fov = ((-view_size2, -view_size2), (view_size2, view_size2))
    problem.visualize_cost_field(fov=fov).visualize_trajectory(traj,
                                                               color="red",
                                                               show=True)

visualize_traj_opt()

print('Tests passed.')

## Three room erratic vacuum world


### Utilities


**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:

from abc import ABC, abstractmethod


class Graph(object):
    """A graph connects nodes (vertices) by edges (links).

    Each edge can also
    have a length associated with it. The constructor call is something like:
        g = Graph({'A': {'B': 1, 'C': 2})
    this makes a graph with 3 nodes, A, B, and C, with an edge of length 1 from
    A to B,  and an edge of length 2 from A to C. You can also do:
        g = Graph({'A': {'B': 1, 'C': 2}, directed=False)
    This makes an undirected graph, so inverse links are also added. The graph
    stays undirected; if you add more links with g.connect('B', 'C', 3), then
    inverse link is also added. You can use g.nodes() to get a list of nodes,
    g.get('A') to get a dict of links out of A, and g.get('A', 'B') to get the
    length of the link from A to B. 'Lengths' can actually be any object at
    all, and nodes can be any hashable object.
    """

    def __init__(self, graph_dict=None, directed=True):
        self.graph_dict = graph_dict or {}
        self.directed = directed
        if not directed:
            self.make_undirected()

    def make_undirected(self):
        """Make a digraph into an undirected graph by adding symmetric
        edges."""
        for a in list(self.graph_dict.keys()):
            for (b, dist) in self.graph_dict[a].items():
                self.connect1(b, a, dist)

    def connect(self, a, b, distance=1):
        """Add a link from a and b of given distance, and also add the inverse
        link if the graph is undirected."""
        self.connect1(a, b, distance)
        if not self.directed:
            self.connect1(b, a, distance)

    def connect1(self, A, B, distance):
        """Add a link from A to B of given distance, in one direction only."""
        self.graph_dict.setdefault(A, {})[B] = distance

    def get(self, a, b=None):
        """Return a link distance or a dict of {node: distance} entries.

        .get(a,b) returns the distance or None; .get(a) returns a dict
        of {node: distance} entries, possibly {}.
        """
        links = self.graph_dict.setdefault(a, {})
        if b is None:
            return links
        else:
            return links.get(b)

    def nodes(self):
        """Return a list of nodes in the graph."""
        s1 = set([k for k in self.graph_dict.keys()])
        s2 = set([k2 for v in self.graph_dict.values() for k2, v2 in v.items()])
        nodes = s1.union(s2)
        return list(nodes)


class Problem(object):
    """The abstract class for a formal problem, based on AIMA.

    You should subclass this and implement abstract methods. Then you
    will create instances of your subclass and solve them with the
    various search functions.
    """

    def __init__(self, initial):
        self.initial = initial

    @abstractmethod
    def actions(self, state):
        """Returns the allowed actions in a given state.

        The result would typically be a list. But if there are many
        actions, consider yielding them one at a time in an iterator,
        rather than building them all at once.
        """
        ...

    @abstractmethod
    def step(self, state, action):
        """Returns the next state when executing a given action in a given
        state.

        The action must be one of self.actions(state).
        """
        ...

    @abstractmethod
    def goal_test(self, state):
        """Checks if the state is a goal."""
        ...


""" [AIMA Figure 4.9]
Eight possible states of the vacumm world
Each state is represented as
   *       "State of the left room"      "State of the right room"   "Room in which the agent
                                                                      is present"
1 - DDL     Dirty                         Dirty                       Left
2 - DDR     Dirty                         Dirty                       Right
3 - DCL     Dirty                         Clean                       Left
4 - DCR     Dirty                         Clean                       Right
5 - CDL     Clean                         Dirty                       Left
6 - CDR     Clean                         Dirty                       Right
7 - CCL     Clean                         Clean                       Left
8 - CCR     Clean                         Clean                       Right
"""
vacuum_world = Graph(
    dict(DDL=dict(Suck=['CCL', 'CDL'], Right=['DDR']),
         DDR=dict(Suck=['CCR', 'DCR'], Left=['DDR']),
         DCL=dict(Suck=['CCL'], Right=['DCR']),
         DCR=dict(Suck=['DCR', 'DDR'], Left=['DCL']),
         CDL=dict(Suck=['CDL', 'DDL'], Right=['CDR']),
         CDR=dict(Suck=['CCR'], Left=['CDL']),
         CCL=dict(Suck=['CCL', 'DCL'], Right=['CCR']),
         CCR=dict(Suck=['CCR', 'CDR'], Left=['CCL'])))


class GraphProblemNonDet(Problem):
    """A graph problem where an action can lead to nondeterministic output i.e.
    multiple possible states."""

    def __init__(self, initial, goal, graph):
        super().__init__(initial)
        self.goal = goal  # a set of states
        self.graph = graph

    def actions(self, A):
        """The actions at a graph node are just its neighbors."""
        return list(self.graph.get(A).keys())

    def step(self, state, action):
        return self.graph.get(state, action)

    def goal_test(self, state):
        return state in self.goal


def AO_DFS(problem):
    """Used when the environment is nondeterministic and completely observable.

    Contains OR nodes where the agent is free to choose any action.
    After every action there is an AND node which contains all possible
    states the agent may reach due to stochastic nature of environment.
    The agent must be able to handle all possible states of the AND node
    (as it may end up in any of them). Returns a conditional plan to
    reach goal state, or failure if the former is not possible.
    """

    # functions used by and_or_search
    def or_search(state, path, problem):
        """returns a plan as a list of actions."""
        if problem.goal_test(state):
            return []
        if state in path:
            return None
        for action in problem.actions(state):
            plan_dict = and_search(problem.step(state, action), path + [
                state,
            ], problem)
            if plan_dict is not None:
                return [action, plan_dict]

    def and_search(states, path, problem):
        """Returns plan in form of dictionary where we take action plan[s] if
        we reach state s."""
        plan_dict = {}
        for s in states:
            plan = or_search(s, path, problem)
            if plan is None:
                return None
            plan_dict[s] = plan
        return plan_dict

    # body of and or search
    return or_search(problem.initial, [], problem)


def sort_graph_dict(data):
    """Normalizes a graph_dict."""
    if isinstance(data, dict):
        for key in data:
            data[key] = sort_graph_dict(data[key])
        return data
    if isinstance(data, list) or isinstance(data, tuple):
        return sorted(data)
    elif isinstance(data, str):
        return data
    else:
        raise ValueError(f"Type {type(data)} not supported")


def graph_dict_md5(data) -> str:
    """Turn Python object into md5 hash.

    Handles nested standard Python types. Used for testing.
    """
    import pprint, hashlib
    return hashlib.md5(pprint.pformat(
        sort_graph_dict(data)).encode('utf-8')).hexdigest()


# A problem that can be passed to AO_DFS; uncomment if you want to use it for testing on Colab
# vacuum_problem = GraphProblemNonDet('DDL', set(['CCL', 'CCR']), vacuum_world)

### Question
Return a dictionary that defines a Graph representing transitions in the three room erratic vacuum world.
    Use letter `M` to denote that the agent is in the middle room.
    For example, `CDCM` means `the left room and right rooms are clean, the middle room is dirty; the robot is in the middle room`.

For reference, our solution is **27** line(s) of code.

In [1]:
def three_room_erratic_world():
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def three_room_erratic_world_test_partial():
    """Checks if `three_room_erratic_world` returns a correct partial answer.

    A full check will be done by comparing the MD5 of the returned
    result, between the submission and our solution.
    """
    ans = three_room_erratic_world()
    assert isinstance(ans, dict), "Incorrect return type"
    ans = sort_graph_dict(ans)
    partial_result = sort_graph_dict(
        dict(
            DDDL=dict(Suck=['CDDL', 'CCDL'], Right=['DDDM']),
            DDDM=dict(Suck=['DCDM', 'CCDM', 'DCCM'],
                      Right=['DDDR'],
                      Left=['DDDL']),
            DDDR=dict(Suck=['DDCR', 'DCCR'], Left=['DDDM']),
            DDCL=dict(Suck=['CDCL', 'CCCL'], Right=['DDCM']),
            DDCM=dict(Suck=['DCCM', 'CCCM'], Right=['DDCR'], Left=['DDCL']),
            DDCR=dict(Suck=['DDCR', 'DDDR'], Left=['DDCM']),
            DCDL=dict(Suck=['CCDL'], Right=['DCDM']),
            DCDM=dict(Suck=['DCDM', 'DDDM'], Right=['DCDR'], Left=['DCDL']),
            DCDR=dict(Suck=['DCCR'], Left=['DCDM']),
            DCCL=dict(Suck=['CCCL'], Right=['DCCM']),
        ))
    for k in partial_result:
        assert k in ans, f"answer should have '{k}' as a key"
    assert {k: ans[k] for k in partial_result
           } == partial_result, "submission does not match the partial result"

three_room_erratic_world_test_partial()



assert graph_dict_md5(three_room_erratic_world()) == "0434567000d839f48149a9d73c8c4a70"
print('Tests passed.')